In [1]:
import re
import pandas as pd
import os
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
import datetime
import json

In [2]:
def getLines(path):
    a = open(path)
    lines = a.readlines()                                       # storing each line of file into a list
    
                                                
    pattern = [{'IS_SPACE': True},
           {'TAG': '-LRB-', 'OP': '?'},
           {'POS': 'NUM'},
           {'TAG': '-RRB-', 'OP': '?'}]
    pattern2 = [{'IS_SPACE': True},{'TAG': 'HYPH'}]             # using spacy pattern to extract fields from file
    
    sp=spacy.load('en_core_web_sm')
    fields =[]
    matcher = Matcher(sp.vocab)
    matcher.add("matching_1",None,pattern,pattern2)             # adding pattern to matcher
    
    start=0
    end=0
    flag=0
    for ab in range(len(lines)):
        lines[ab] = re.sub("Â£","&#163",lines[ab])              # replacing the currency with html code
        lines[ab] = re.sub("Â€","&#8364",lines[ab])
        lines[ab] = re.sub("â‚¬","&#8364",lines[ab])
        lines[ab] = re.sub("Â$","&#36",lines[ab])
        sen=sp(lines[ab])
        
        
        if "Â®" in lines[ab] or "Notes to"\
        in lines[ab] or "For the" in lines[ab] or "The" in lines[ab]\
        or "Statements" in lines[ab] or "NOTES" in lines[ab]\
        or "STATEMENTS" in lines[ab]\
        or "Number" in lines[ab]:                               # stop when no more fields are there
            break
        if re.fullmatch("Notes",lines[ab].strip()):             #"Notes" in lines[ab] and ab>5:
            break
        
        matches = matcher(sen)                                  # using matcher for extractinhg only required fields from the file
        if(len(matches)>0):
            if flag==0:
                flag=1
                start=ab
            end=ab

    lines=lines[start:end+1]  
    for a in range(len(lines)):                                # removing leading and trailing spaces from the extracted lines
        lines[a]=lines[a].strip()
    #     print(lines)
    lines[0]=re.sub('\s\s+',' ',lines[0])                      # subsituting extra space in the first line( "Notes 2019 2018") with single space
    for l in range(1,len(lines)):                              # replacing extra space in rest of the lines with \s
        lines[l] = re.sub('\s\s+','\s',lines[l])
    
    return lines


In [3]:
def getKeyValue(lines):
    key = []
    value = []
    col = -1 
    st=0
    
    
    lin = lines[0].split(" ");

    if lin[0]=="Notes" or lin[0]=="Note": # if notes is present in first line
        st=1
        cols_length=len(lin)
    else:
        cols_length=len(lin)+1   # if notes is not present then for rest of the line number of columns is assigend 
    
    # for loop extracts the last four letter to check for the years
    for a in range(st,len(lin)):
        val =lin[a]
        try:
            val=int(val[-4:])
        except:
            pass
        
        if val==2019: # if 2019 is there than column for 2019 is aasgned to col
            col=a
    
    # if the year 2019 is not found and "Notes" is not present in header than updating col value 
    if col!=-1:
        if lin[0]=="Notes" or lin[0]=="Note" :
            pass
        else:
            col=col+1 

    #start variable is used to know the line from where processing should start
    start=1
    if "&#36" in lines[1] or "&#8364" in lines[1] or "&#163" in lines[1]:
        start=2
    a=start
    
    # processing
    while a<len(lines):
        spl = lines[a].split("\s")
        
        # if year 2019 is in the file
        if col!=-1:
            # checking if first col of spl is integer
            if re.fullmatch(r'\d+,*\.*,*\d*',spl[0]) or re.fullmatch(r'\(\d+,*\.*,*\d*\)',spl[0]) or re.fullmatch(r'-',spl[0]):
                key.append("nan")
                
                if len(spl)==cols_length:
                    value.append(spl[col])
                else:
                    try:
                        value.append(spl[col-1])
                    except:
                        value.append("nan")
            # checking if the length of spl is 1
            elif len(spl)==1 and spl[0]!="":
                temp_a=a+1
                try:
                    temp_spl=lines[temp_a].split("\s")
                except:
                    pass
        
                # checking if the next line to spl start with small letter if yes than this line is appended to spl
                if re.match('^[a-z]',temp_spl[0]):
                    
                
                    key.append((spl[0]+" "+temp_spl[0]))
                    try:
                        
                        if len(temp_spl)==cols_length:
                            value.append(temp_spl[col])
                        elif len(temp_spl)==cols_length+1:
                            value.append(temp_spl[col+1])
                        else:
                            value.append(temp_spl[col-1])
                    except:
                        value.append("nan")
                    a=a+1
                
                # checking if the next line to spl contains alphabets, 
                # if yes add all the columns of this line to spl till last alphabet is encountered
                # eg : 
                elif re.search('[a-zA-Z]',lines[temp_a].replace("\s","")):
                    lst=0
                    # for loop to get last column in new line which has alphabet eg: "100 100 Ordinary Share  10 10"
                    for cha in reversed(range(len(temp_spl))):
                       
                        if re.search('[a-zA-Z]',temp_spl[cha]):
                         
                            lst = cha
                            break

                    temp_key=""
                    # iterating to get the key
                    for lst_val in range(lst+1):
                        temp_key = temp_key+" "+temp_spl[lst_val]
                    
                    key.append(spl[0])
                    value.append("nan")
                    key.append(temp_key.strip())

                    try:

                        if len(temp_spl)-lst==cols_length:
                            value.append(temp_spl[col])
                        elif len(temp_spl)-lst==cols_length+1:
                            value.append(temp_spl[col+1])
                        else:
                            value.append(temp_spl[col])
                    except:
                        value.append("nan")

                    a=a+1
                # checking if the next line only contains number s or - or "(numbers)"   
                elif re.fullmatch(r'\d+,*\.*,*\d*',temp_spl[0]) or re.fullmatch(r'\(\d+,*\.*,*\d*\)',temp_spl[0]) or re.fullmatch(r'-',temp_spl[0]):
                    try:
                        
                        tp = lines[temp_a+1].split("\s")
               
                        # checking if the line next to this contians only "year" else continue
                        if len(tp)==1 and tp[0]=="year":
                            key.append((spl[0]+" year"))                         # adding "year" to the key

                            if len(temp_spl)==cols_length:

                                value.append(temp_spl[col])
                            else:
                                value.append(temp_spl[col-1])
                            a=a+2                                               # increment a by two
                    except:
                        pass
                else:

                    if spl[0]!="":
                        key.append(spl[0])
                        value.append("nan")
            elif len(spl)==cols_length: # case when no notes value

                key.append(spl[0])
                value.append(spl[col])
            elif len(spl)==(cols_length+1): # case when note value is there

                key.append(spl[0])
                value.append(spl[col+1])
            else:

                if spl[0]!="":
                    key.append(spl[0])
                    try:
                        value.append(spl[col])
                    except:
                        value.append("nan")
        else:
            if re.fullmatch(r'\d+\.*\d*',spl[0]) or re.fullmatch(r'\(\d+\.*\d*\)',spl[0]) or re.fullmatch(r'-',spl[0]):

                pass
            else:

                key.append(spl[0])
                value.append("nan")
        a=a+1
    
    return key,value
    

In [4]:
def getDict(key,value):
    values= []
    keys=[]

    for a in range(len(key)):
        if re.match(r'\d',value[a]):    # checking if the value is a umber

            values.append(value[a].replace(',','').split(" ")[0].split(".")[0]) # replacing "," , "."
            keys.append(key[a])
        elif re.match(r'[\(\d\)]',value[a]): # checking if number is inside brackets
            # replacinf "," , "." and "(" ,")"
            val= value[a].replace(",","")
            val = val.replace("(","-")
            val= val.replace(")","")
            values.append(val.split(" ")[0].split(".")[0])
            keys.append(key[a])
        elif re.match(r'-',value[a]):   # checking is "- " is there
            values.append('-')
            keys.append(key[a])
        else:
            keys.append(key[a])
            values.append(value[a])

    return json.dumps(dict(zip(keys,values)))

In [5]:
i=0
filename=[]
expected_values=[]
# iterating through each file in directory
for file in os.listdir("D:\\HCL ML Challenge\\HCL ML Challenge Dataset\\"):
    path = os.path.join("D:\\HCL ML Challenge\\HCL ML Challenge Dataset\\",file)
    data = pd.DataFrame()

    if file.endswith(".txt"):
        d=getLines(path)              # calling getLines() to get only required lines from the file
        key,value=getKeyValue(d)      # calling getKeyValue() to get the keys and values list
        d = getDict(key,value)        # calling getDict() to get the final dictonaru
        filename.append(file.split(".")[0])
        expected_values.append(d)

# saving the filename and expected_value as pandas dataframe
final_dataframe =pd.DataFrame(list(zip(filename,expected_values)),columns=["Filename","Extracted Values"]) 
# saving the dataframe to csv
final_dataframe.to_csv("C:\\Users\\Akash Srivastava\\Desktop\\result.csv",index = False)